# CT Reconstruction Pipeline Notebook

This notebook guides through the process of reconstructing 3D volume data from 2D projection images using the modules developed in the `src` directory.

## 1. Setup and Imports

Import necessary libraries and the custom modules.

In [1]:
import numpy as np
from pathlib import Path
import logging
import matplotlib.pyplot as plt

# Ensure the src directory is in the Python path
# If running the notebook from the project root directory, this should work:
import sys
if './src' not in sys.path:
    sys.path.append('./src')

# Import custom modules
import config as cfg
import utils
import data_io
import preprocess
import reconstruction
import visualization

# Configure matplotlib for inline display in Jupyter
%matplotlib inline

## 2. Configuration Loading

Load parameters from the configuration file.

In [2]:
# Define the path to the configuration file
CONFIG_FILE = Path("./config/default_params.yaml")

# Load configuration
config = cfg.load_config(CONFIG_FILE)

if config:
    print("Configuration loaded successfully:")
    # Pretty print the config dictionary
    import json
    print(json.dumps(config, indent=2))
else:
    raise RuntimeError(f"Failed to load configuration from {CONFIG_FILE}")

2025-05-03 14:34:54,201 - config - INFO - Successfully loaded configuration from: config/default_params.yaml


Configuration loaded successfully:
{
  "data_base_dir": "./data/",
  "dataset_name": "dataset_1",
  "preprocess_epsilon": 1e-06,
  "reconstruction_algorithm": "fbp",
  "fbp_filter": "ramp",
  "output_slice_size": null,
  "results_base_dir": "./results/",
  "output_prefix": "recon",
  "log_level": "INFO",
  "log_file": "pipeline.log"
}


## 3. Logging Setup

Configure logging based on settings from the config file.

In [3]:
# Get logging parameters from config
log_level_str = config.get('log_level', 'INFO').upper()
log_level = getattr(logging, log_level_str, logging.INFO)
log_file_config = config.get('log_file', None) # Can be null/None in YAML

# Construct full log file path if specified
log_file_path = None
if log_file_config:
    # Assume log file path is relative to results base dir or project root
    # Here, let's put it in the results base directory
    results_base = Path(config.get('results_base_dir', './results/'))
    log_file_path = results_base / log_file_config

# Setup logging using the utility function
utils.setup_logging(level=log_level, log_file=log_file_path)

logging.info("Logging initialized for the notebook.")
logging.debug(f"Using configuration: {config}")

2025-05-03 14:34:54 - root - INFO - Logging configured. Level: INFO. Outputting to console and file: results/pipeline.log
2025-05-03 14:34:54 - root - INFO - Logging initialized for the notebook.


## 4. Define Data Paths

Construct the specific paths for the dataset being processed based on the configuration.

In [4]:
data_base_dir = Path(config.get('data_base_dir', './data/'))
dataset_name = config.get('dataset_name', 'default_dataset')
dataset_dir = data_base_dir / dataset_name

proj_dir = dataset_dir / "projections"
flat_dir = dataset_dir / "flats"
dark_dir = dataset_dir / "darks"
metadata_file = dataset_dir / "metadata.txt" # Or whatever the metadata file is named

logging.info(f"Processing dataset: {dataset_name}")
logging.info(f"Projections directory: {proj_dir}")
logging.info(f"Flats directory: {flat_dir}")
logging.info(f"Darks directory: {dark_dir}")
logging.info(f"Metadata file: {metadata_file}")

2025-05-03 14:34:54 - root - INFO - Processing dataset: dataset_1
2025-05-03 14:34:54 - root - INFO - Projections directory: data/dataset_1/projections
2025-05-03 14:34:54 - root - INFO - Flats directory: data/dataset_1/flats
2025-05-03 14:34:54 - root - INFO - Darks directory: data/dataset_1/darks
2025-05-03 14:34:54 - root - INFO - Metadata file: data/dataset_1/metadata.txt


## 5. Data Loading

Load the projection images, flat fields, dark fields, and metadata (angles).

In [5]:
# Load data using data_io module
projections = data_io.load_tiff_sequence(proj_dir)
flats = data_io.load_tiff_sequence(flat_dir)
darks = data_io.load_tiff_sequence(dark_dir)
angles_rad = data_io.load_metadata(metadata_file)

# Basic checks
if projections is None or flats is None or darks is None or angles_rad is None:
    raise ValueError("Failed to load one or more data components. Check logs and paths.")

logging.info(f"Loaded projections shape: {projections.shape}")
logging.info(f"Loaded flats shape: {flats.shape}")
logging.info(f"Loaded darks shape: {darks.shape}")
logging.info(f"Loaded angles shape: {angles_rad.shape}")

# Verify consistency
if projections.shape[0] != len(angles_rad):
    raise ValueError("Number of projections does not match number of angles!")
if projections.shape[1:] != flats.shape[1:] or projections.shape[1:] != darks.shape[1:]:
    raise ValueError("Image dimensions (height, width) do not match between projections, flats, and darks!")

2025-05-03 14:34:58 - root - INFO - Found 361 TIFF files in data/dataset_1/projections. Loading sequence...
2025-05-03 14:35:08 - root - INFO - Successfully loaded image stack with shape: (361, 2368, 2240)
2025-05-03 14:35:08 - root - ERROR - Error: Provided path 'data/dataset_1/flats' is not a valid directory.
2025-05-03 14:35:08 - root - ERROR - Error: Provided path 'data/dataset_1/darks' is not a valid directory.
2025-05-03 14:35:08 - root - INFO - Read from metadata: NumberImages=360, AngleFirst=0.0, AngleInterval=1.0
2025-05-03 14:35:08 - root - INFO - Successfully calculated 360 angles (radians) from metadata.


ValueError: Failed to load one or more data components. Check logs and paths.

## 6. Data Exploration (Optional)

Visualize some of the raw loaded data.

In [ ]:
# Inside Cell 6
avg_flat = preprocess.average_images(flats) if flats is not None else None
avg_dark = preprocess.average_images(darks) if darks is not None else None

if avg_flat is not None and avg_dark is not None:
    visualization.plot_comparison(avg_flat, avg_dark, title1="Average Flat Field", title2="Average Dark Field", main_title="Calibration Frames", colorbar_label="Counts")
    plt.show()
else:
    logging.warning("Average flats/darks cannot be visualized as one or both were not loaded.")

## 7. Preprocessing

Apply dark subtraction, flat-field correction, and negative logarithm.

In [ ]:
# Get epsilon from config
epsilon = config.get('preprocess_epsilon', 1e-6)

# Perform preprocessing
attenuation_sinograms = preprocess.preprocess_data(projections, flats, darks, epsilon=epsilon)

if attenuation_sinograms is None:
    raise RuntimeError("Preprocessing failed. Check logs.")

logging.info(f"Preprocessing complete. Attenuation data shape: {attenuation_sinograms.shape}")

## 8. Preprocessing Visualization (Optional)

Visualize a sinogram slice after preprocessing.

In [ ]:
# Inside Cell 8
if attenuation_sinograms is not None:
    height = attenuation_sinograms.shape[1]
    middle_slice_idx = height // 2
    sinogram_slice = attenuation_sinograms[:, middle_slice_idx, :]
    # Check if correction was likely skipped (by comparing with raw projections)
    # A more robust check might involve a flag returned from preprocess_data
    was_corrected = not np.allclose(attenuation_sinograms, projections.astype(np.float32))

    plot_title = f"Sinogram (Slice {middle_slice_idx})"
    colorbar_lbl = "Value"
    if was_corrected:
        plot_title = f"Preprocessed Sinogram (Slice {middle_slice_idx})"
        colorbar_lbl = "Attenuation"
    else:
         plot_title = f"Raw Sinogram (Slice {middle_slice_idx} - No Correction)"
         colorbar_lbl = "Counts / Intensity"

    visualization.plot_sinogram(sinogram_slice, title=plot_title, colorbar_label=colorbar_lbl)
    plt.show()

## 9. Reconstruction

Perform tomographic reconstruction using the chosen algorithm (e.g., FBP).

In [ ]:
# Get reconstruction parameters from config
algorithm = config.get('reconstruction_algorithm', 'fbp').lower()
fbp_filter = config.get('fbp_filter', 'ramp')
output_size = config.get('output_slice_size', None) # Can be null/None

reconstructed_volume = None
if algorithm == 'fbp':
    logging.info(f"Performing FBP reconstruction with filter: {fbp_filter}")
    reconstructed_volume = reconstruction.reconstruct_fbp(
        attenuation_sinograms,
        angles_rad,
        filter_name=fbp_filter,
        output_size=output_size
    )
elif algorithm == 'sirt':
    # Placeholder for future SIRT implementation
    # sirt_iterations = config.get('sirt_iterations', 100)
    logging.warning("SIRT reconstruction not yet implemented in reconstruction.py module.")
    # reconstructed_volume = reconstruction.reconstruct_sirt(attenuation_sinograms, angles_rad, iterations=sirt_iterations)
else:
    logging.error(f"Unsupported reconstruction algorithm specified: {algorithm}")

if reconstructed_volume is None and algorithm == 'fbp': # Check if FBP was attempted but failed
     raise RuntimeError("Reconstruction failed. Check logs.")
elif reconstructed_volume is not None:
    logging.info(f"Reconstruction complete. Volume shape: {reconstructed_volume.shape}")

## 10. Result Visualization

Display some slices from the reconstructed 3D volume.

In [ ]:
if reconstructed_volume is not None:
    num_recon_slices = reconstructed_volume.shape[0]

    # Show middle slice
    middle_recon_slice_idx = num_recon_slices // 2
    visualization.plot_slice(reconstructed_volume[middle_recon_slice_idx],
                             title=f"Reconstructed Slice {middle_recon_slice_idx} ({algorithm.upper()})",


## 11. Saving Results

Save the reconstructed volume as a stack of TIFF slices.

In [ ]:
if reconstructed_volume is not None:
    # Define output directory and prefix
    results_base_dir = Path(config.get('results_base_dir', './results/'))
    output_dir_name = f"{dataset_name}_recon_{algorithm}" # e.g., dataset_1_recon_fbp
    output_directory = results_base_dir / output_dir_name
    output_prefix = config.get('output_prefix', 'recon_slice')

    logging.info(f"Saving reconstructed volume to: {output_directory}")
    data_io.save_tiff_stack(reconstructed_volume, output_directory, file_prefix=output_prefix)
    logging.info("Volume saving complete.")
else:
    logging.warning("Skipping result saving as reconstruction was not successful or not performed.")

--- Pipeline End ---